In [21]:
import pandas as pd
df = pd.read_csv('car_price_dataset.csv') 
df.columns = df.columns.str.strip()
print(df.columns)
df

#visualização inicial do DataSet

Index(['Brand', 'Model', 'Year', 'Engine_Size', 'Fuel_Type', 'Transmission',
       'Mileage', 'Doors', 'Owner_Count', 'Price'],
      dtype='object')


,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867
...,...,...,...,...,...,...,...,...,...,...
9995,Kia,Optima,2004,3.7,Diesel,Semi-Automatic,5794,2,4,8884
9996,Chevrolet,Impala,2002,1.4,Electric,Automatic,168000,2,1,6240
9997,BMW,3 Series,2010,3.0,Petrol,Automatic,86664,5,1,9866
9998,Ford,Explorer,2002,1.4,Hybrid,Automatic,225772,4,1,4084


In [ ]:
df.info
df.isnull().sum()
#verificando se há algum valor nulo para ser tratado

Brand           0
Model           0
Engine_Size     0
Fuel_Type       0
Transmission    0
Mileage         0
Doors           0
Owner_Count     0
Price           0
Car_Age         0
dtype: int64

In [ ]:
model_counts = df["Model"].value_counts()
model_counts_df = model_counts.reset_index()
model_counts_df.columns = ["Model", "Amount"]
print(model_counts_df)
#verificando quantos modelos e a quantidade deles

model_counts.mean()
#vendo a média de carrros por modelo


       Model  Amount
0     Accord     365
1     Fiesta     363
2     Impala     355
3   5 Series     353
4     Tiguan     352
5      Focus     351
6         Q5     350
7         A4     350
8    Elantra     347
9       Golf     339
10        A3     338
11    Optima     336
12     Civic     336
13  3 Series     335
14  Explorer     334
15    Malibu     333
16    Tucson     329
17    Passat     329
18       Rio     327
19   Corolla     325
20      RAV4     324
21     Camry     321
22   E-Class     321
23    Sonata     319
24   Equinox     315
25       GLA     315
26  Sportage     313
27        X5     311
28      CR-V     308
29   C-Class     306


333.3333333333333

In [22]:
from datetime import datetime

current_year = datetime.now().year

df['Age'] = current_year - df['Year']

print(df[['Year', 'Age']].head())

   Year  Age
0  2020    5
1  2012   13
2  2020    5
3  2023    2
4  2003   22
